In [1]:
from graph import read_data, plot_data, read_dsc


In [2]:
data = read_dsc('dsc_data\\empty_pans')
data
# fig = plot_data(data)
# fig.show()

,sig1,sig2,sig3,sig4,sig5,sig6,sig7
0,0.004333,26.44116,0.013093,-5.051737,-0.293197,-1.851977,100.00640
1,0.004333,26.44151,0.013106,-5.051238,-0.293128,-1.851540,100.04970
2,0.012667,26.44248,0.013086,-5.050625,-0.293043,-1.851003,99.95844
3,0.021000,26.44013,0.013132,-5.050235,-0.292989,-1.850662,100.02150
4,0.029333,26.44353,0.013158,-5.049896,-0.292942,-1.850364,100.00830
...,...,...,...,...,...,...,...
3638,30.312660,324.04340,-0.014832,-9.104000,-0.533024,-5.538556,99.97338
3639,30.321000,324.12500,-0.014822,-9.105624,-0.533151,-5.540182,100.01180
3640,30.329340,324.20760,-0.014845,-9.107160,-0.533270,-5.541732,100.01050
3641,30.337670,324.29370,-0.014856,-9.108720,-0.533388,-5.543269,100.00920
